# Trade Simulator Code

This is the cell block that allows for the sim to run. The interactive front-end is produced at the bottom of the cell, which has various buttons and settings to control the simulation as desired.  

In [17]:
# import libraries
%matplotlib agg
import random
import time
import threading
import os
import io
import warnings
import mplfinance as mpf
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import ipywidgets as widgets
from prettytable import PrettyTable
from ipywidgets import AppLayout
from IPython.display import display
from collections import deque

# Set the pandas option to opt into the future behavior, and suppress any SettingWithCopyWarning and FutureWarning messages
pd.set_option('future.no_silent_downcasting', True)
pd.options.mode.chained_assignment = None 
warnings.simplefilter(action='ignore', category=FutureWarning)

# import local scripts
import dynamicrisk as dr
import botlogic as bl
import botorder as bo
import participantsetup as ps

# Creating an "output" folder which is used as a dependency of this code in order to create/update visualisations
output_path = 'output'
if not os.path.exists(output_path):
    os.makedirs(output_path)

# Calculating how to scale time, if applicable
def sim_to_real_time (sim_time):
    global time_scalar, start_time, sim_duration
    if time_scalar == 1:
        real_time = sim_time
    else:
        elapsed = sim_time.timestamp() - start_time
        elapsed_real_world = elapsed * time_scalar
        real_time = start_time + elapsed_real_world
        real_time = dt.datetime.fromtimestamp(real_time)
    return real_time

# Market infrastructure
def buy_orderbook_append (trader_id, timestamp, quantity, price): 
    global buy_orderbook, orderbook_log
    if len(orderbook_log) < 1:
        order_id = 1
    else:
        order_id = int(orderbook_log.iloc[-1,0] + 1)

    # appending order to the live buy orderbook
    timestamp = pd.to_datetime(timestamp)
    new_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price}
    to_orderbook = pd.Series(new_order)
    buy_orderbook = pd.concat([buy_orderbook, to_orderbook.to_frame().T], ignore_index=True)
    buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
    buy_orderbook["Timestamp"] = pd.to_datetime(buy_orderbook["Timestamp"])

    # appending order to the orderbook log
    new_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price, "Side": "Buy", "Status": "Open", "Update_Timestamp": timestamp, "Version": 1}
    to_log = pd.Series(new_order)
    orderbook_log = pd.concat([orderbook_log, to_log.to_frame().T], ignore_index=True)

def sell_orderbook_append (trader_id, timestamp, quantity, price):
    global sell_orderbook, orderbook_log 
    if len(orderbook_log) < 1:
        order_id = 1
    else:
        order_id = int(orderbook_log.iloc[-1,0] + 1)

    # appending order to the live sell orderbook
    timestamp = pd.to_datetime(timestamp)
    new_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price}
    to_orderbook = pd.Series(new_order)
    sell_orderbook = pd.concat([sell_orderbook, to_orderbook.to_frame().T], ignore_index=True)
    sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
    sell_orderbook["Timestamp"] = pd.to_datetime(sell_orderbook["Timestamp"])

    # appending order to the orderbook log
    new_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price, "Side": "Sell", "Status": "Open", "Update_Timestamp": timestamp, "Version": 1}
    to_log = pd.Series(new_order)
    orderbook_log = pd.concat([orderbook_log, to_log.to_frame().T], ignore_index=True)

def orderbook_amend (order_id, trader_id, timestamp, quantity, price, side):
    global buy_orderbook, sell_orderbook
    if side == "Buy":
        update_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price}
        update_orderbook = pd.Series(update_order)
        buy_orderbook = pd.concat([buy_orderbook, update_orderbook.to_frame().T], ignore_index=True)
        buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
        buy_orderbook["Timestamp"] = pd.to_datetime(buy_orderbook["Timestamp"])
    elif side == "Sell":
        update_order = {"Order_ID": order_id, "Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price}
        update_orderbook = pd.Series(update_order)
        sell_orderbook = pd.concat([sell_orderbook, update_orderbook.to_frame().T], ignore_index=True)
        sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
        sell_orderbook["Timestamp"] = pd.to_datetime(sell_orderbook["Timestamp"])

def candle_recalc(candle_length):
    global transaction_log
    str_candle_length = "".join((str(candle_length),'s'))
    recalc_df = transaction_log.copy()
    recalc_df["timestamp"] = recalc_df['Timestamp'].dt.floor(str_candle_length)
    recalc_df = recalc_df.groupby("timestamp").agg({
        "Price" : ['first', 'max', 'min','last'],
        "Quantity" : 'sum'
    })
    recalc_df.columns = ['open', 'high', 'low', 'close', 'volume']
    columns_to_convert = ['open', 'high', 'low', 'close']
    recalc_df[columns_to_convert] = recalc_df[columns_to_convert].round(2).astype(float)
    recalc_df["volume"] = recalc_df["volume"].astype(int)
    return recalc_df

current_window_transactions = deque()
def transaction_append (timestamp, buy_order_id, buy_id, sell_order_id, sell_id, quantity, price, aggressor_side, aggressor_id):
    global transaction_log, candlestick_df, time_elapsed, current_window_start, current_window_transactions, candle_length, time_scalar
    quantity = int(quantity)
    price = float(price)
    transaction = pd.Series({
        "Timestamp" : timestamp, 
        "Buy_Side_Order_ID" : int(buy_order_id),
        "Buy_Side_Trader_ID" : int(buy_id), 
        "Sell_Side_Order_ID" : int(sell_order_id),
        "Sell_Side_Trader_ID" : int(sell_id), 
        "Quantity" : quantity, 
        "Price" : price, 
        "Aggressor" : aggressor_side,
        "Aggressor_ID" : aggressor_id})
    transaction_log = pd.concat([transaction_log, transaction.to_frame().T], ignore_index=True)
    transaction_log["Timestamp"] = pd.to_datetime(transaction_log["Timestamp"])
    transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)

    if time_scalar == 1:
        if time_elapsed < 180:
            candle_length = 15
        elif time_elapsed >= 180 and time_elapsed < 300:
            candle_length = 30
            candlestick_df = candle_recalc(candle_length)
        elif time_elapsed >= 300 and time_elapsed < 900:
            candle_length = 60
            candlestick_df = candle_recalc(candle_length)
        elif time_elapsed >= 900:
            candle_length = 120
            candlestick_df = candle_recalc(candle_length)
    else:
        candle_length = 1800 # if time is being scaled, each candle will be worth a half hour
    candle_length = dt.timedelta(seconds=candle_length)

    if timestamp < current_window_start + candle_length:
        current_window_transactions.append({'price': price, 'volume': quantity})
    else:
        # Calculate OHLC and volume for the completed window
        open_price = current_window_transactions[0]['price']
        high_price = max(tx['price'] for tx in current_window_transactions)
        low_price = min(tx['price'] for tx in current_window_transactions)
        close_price = current_window_transactions[-1]['price']
        total_volume = sum(tx['volume'] for tx in current_window_transactions)

        # Add the completed window's data to the DataFrame
        candlestick = pd.Series({
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'volume': total_volume
        })
        candle_df = candlestick.to_frame().T
        candle_df.index = [current_window_start]
        candlestick_df = pd.concat([candlestick_df, candle_df])
        # Start a new window
        current_window_start = current_window_start + candle_length
        current_window_transactions = deque([{'price': price, 'volume': quantity}])

    # Update the last row with the current transaction
    if not candlestick_df.empty:
        open_price = current_window_transactions[0]['price']
        high_price = max(tx['price'] for tx in current_window_transactions)
        low_price = min(tx['price'] for tx in current_window_transactions)
        close_price = current_window_transactions[-1]['price']
        total_volume = sum(tx['volume'] for tx in current_window_transactions)

        candlestick_df.iloc[-1] = {
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'volume': total_volume
        }

    else:
        open_price = current_window_transactions[0]['price']
        high_price = max(tx['price'] for tx in current_window_transactions)
        low_price = min(tx['price'] for tx in current_window_transactions)
        close_price = current_window_transactions[-1]['price']
        total_volume = sum(tx['volume'] for tx in current_window_transactions)
        candlestick = pd.Series({
            'open': open_price,
            'high': high_price,
            'low': low_price,
            'close': close_price,
            'volume': total_volume
        })
        candle_df = candlestick.to_frame().T
        candle_df.index = [current_window_start]
        candlestick_df = pd.concat([candlestick_df, candle_df ])


# this updates the log with any executions or cancellations 
def h_log_update(input, action, action_time):
    global orderbook_log
    update_order_index = orderbook_log[(orderbook_log["Order_ID"] == input["Order_ID"]) & (orderbook_log["Quantity"] == input["Quantity"])].index
    update_cols = ["Status", "Update_Timestamp"]
    if action == 'passive_order_executed':
        orderbook_log.loc[update_order_index, update_cols] = ["Passive Execution", action_time]
    elif action == "aggressive_order_executed":
        orderbook_log.loc[update_order_index, update_cols] = ["Aggressive Execution", action_time]
    elif action == 'order_cancelled_price':
        orderbook_log.loc[update_order_index, update_cols] = ["Cancelled - on price", action_time]
    elif action == 'order_cancelled_time':
        orderbook_log.loc[update_order_index, update_cols] = ["Cancelled - on time", action_time]
    orderbook_log["Update_Timestamp"] = pd.to_datetime(orderbook_log["Update_Timestamp"])

# this updates the log with any partial fills (amendments are not a feature)
def h_log_amend (input, q_update, side, fill_type):
    global orderbook_log
    order_id = int(input["Order_ID"])
    order_qty = int(input["Quantity"])
    
    amended_time = sim_to_real_time(dt.datetime.now())
    # boolean condition to retrieve the version number of the amended order
    stale_order_index = orderbook_log[(orderbook_log["Order_ID"] == order_id) & (orderbook_log["Quantity"] == order_qty)].index
    update_cols = ["Status", "Update_Timestamp"]

    # determine update message based on type of order fill
    if fill_type == "Aggressive":
        orderbook_log.loc[stale_order_index, update_cols] = ["Aggressive Partial Fill", amended_time]
    elif fill_type == "Passive":
        orderbook_log.loc[stale_order_index, update_cols] = ["Passive Partial Fill", amended_time]
    
    try:
        version_count = orderbook_log.loc[stale_order_index, "Version"].values[0]
        version_count = int(version_count + 1)
    except:
        version_count = 2
    child_order = {
        "Order_ID": order_id, 
        "Trader_ID" : int(input["Trader_ID"]), 
        "Timestamp" : amended_time, 
        "Quantity" : q_update, 
        "Price" : float(input["Price"]), 
        "Side": side, 
        "Status": "Open", 
        "Update_Timestamp": amended_time, 
        "Version": version_count
    }
    child_order_pd = pd.Series(child_order)
    orderbook_log = pd.concat([orderbook_log, child_order_pd.to_frame().T], ignore_index=True)
    orderbook_log["Timestamp"] = pd.to_datetime(orderbook_log["Timestamp"])
    orderbook_log["Update_Timestamp"] = pd.to_datetime(orderbook_log["Update_Timestamp"])
    orderbook_log.sort_values(by=["Order_ID", "Version"], ascending=[True, True], inplace=True)

# Pulling key figures
class market_key_figures:
    def __init__(self):
        global transaction_log, buy_orderbook, sell_orderbook, orderbook_log
        # tests to calculate figs from transaction log
        try:
            transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)
            self.market_price = transaction_log["Price"].iloc[-1]
            self.open_price = transaction_log["Price"].iloc[0]
            self.abs_price_mvmt = round(self.market_price - self.open_price,2)
            self.per_price_mvmt = round((self.abs_price_mvmt/self.open_price)*100,2)

            transaction_log.sort_values(by=["Price"], ascending=[True], inplace = True)
            self.price_max = transaction_log["Price"].iloc[-1]
            self.price_min = transaction_log["Price"].iloc[0]
            self.price_range = round(self.price_max - self.price_min,2)

            transaction_log.sort_values(by=["Timestamp"], ascending=[True], inplace=True)
            self.transact_test = "pass"
        except:
            self.transact_test = "fail"

        # tests to calculate figs from orderbook
        try: 
            buy_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[False, True], inplace=True)
            self.best_bid = buy_orderbook['Price'].iloc[0]
            self.buy_orderbook_test = "b_pass"
        except:
            self.buy_orderbook_test = "b_fail"

        try:
            sell_orderbook.sort_values(by=["Price", "Timestamp"], ascending=[True, True], inplace=True)
            self.best_ask = sell_orderbook['Price'].iloc[0]
            self.sell_orderbook_test = "s_pass"
        except:
            self.sell_orderbook_test = "s_fail"
      
        # combining these tests to produce a single outcome result 
        if self.transact_test == "pass" and (self.buy_orderbook_test == "b_pass" and self.sell_orderbook_test == "s_pass"):
            self.key_figs_test = "pass"
        elif self.transact_test == "fail" and (self.buy_orderbook_test == "b_pass" and self.sell_orderbook_test == "s_pass"):
            self.key_figs_test = "o_semipass"
        elif self.transact_test == "pass" and (self.buy_orderbook_test == "b_fail" or self.sell_orderbook_test == "s_fail"):
            self.key_figs_test = "t_semipass"

        #elif self.transact_test == "fail" and (self.buy_orderbook_test == "b_fail" or self.sell_orderbook_test == "s_fail"):
        else: 
            self.key_figs_test = "fail"

# Order Matching & transaction log
def orderbook_matching (input_order, bot):
    global orderbook_log, df_participants, transaction_log, buy_orderbook, sell_orderbook
    if len(orderbook_log) < 1:
        log_order_id = 1
    else:
        log_order_id = orderbook_log.iloc[-1,0] + 1

    def bot_crediting (transaction_price, transaction_qty, buy_ID, sell_ID, price_diff, aggressor, bot):
        global df_participants
        transaction_val = transaction_price * transaction_qty
        if aggressor == "buy":
            # crediting aggressor to the bot profile
            bot.iloc[1] += transaction_qty   
            # crediting profile to the sell participant
            df_participants.loc[df_participants["Trader_ID"] == sell_ID, "Wealth"] += round(transaction_val, 2)  
        elif aggressor == "sell":
            # crediting profile to the bot profile
            df_participants.loc[df_participants["Trader_ID"] == buy_ID, "Asset"] += transaction_qty
            # crediting aggressor to the sell participant
            bot.iloc[2] += round(transaction_val, 2)    
        if price_diff > 0 and aggressor == "buy":
            val_diff = price_diff * transaction_qty
            bot.iloc[2] += round(val_diff, 2)
        return bot

    # Not the cleanest code, but it works for now. 
    def bid_price_check (order_price, best_ask):
        if order_price >= best_ask:
            check = "True"
        else:
            check = "False"
        return check
    def ask_price_check (order_price, best_bid):
        if order_price <= best_bid:
            check = "True"
        else:
            check = "False"
        return check
    
    key_figs =market_key_figures()
    order_price = float(round(input_order["Price"],2))
    bid_ask_flag = input_order["Flag"]
    order_quantity = int(input_order["Quantity"])
    input_order_id = int(input_order["Trader_ID"])
    timestamp = sim_to_real_time(dt.datetime.now())

    try:
        bprice_check = bid_price_check(order_price, key_figs.best_ask)
    except:
        bprice_check = "False"
    
    try:
        aprice_check = ask_price_check(order_price, key_figs.best_bid)
    except:
        aprice_check = "False"

    # any orders that do not match the best bid are appended to the sell orderbook
    if (bid_ask_flag == "ask" and aprice_check == "False") and order_quantity > 0:
        sell_orderbook_append(input_order_id, timestamp, order_quantity, order_price)
        # any orders that do not match the best ask are appended to the buy orderbook
    elif (bid_ask_flag == "bid" and bprice_check == "False") and order_quantity > 0:
        buy_orderbook_append(input_order_id, timestamp, order_quantity, order_price)
    elif bid_ask_flag == "bid" and bprice_check == "True":
        # appending to the orderbook log
        log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Timestamp" : timestamp, 
                "Quantity" : order_quantity, 
                "Price" : order_price, 
                "Side" : "Buy", 
                "Status" : "Open", 
                "Update_Timestamp" :timestamp,
                "Version" : 1
            }
        log_order = pd.Series(log)
        orderbook_log = pd.concat([orderbook_log, log_order.to_frame().T], ignore_index=True)

        # Running a while loop to execute the order against as many eligible orders 
        while order_quantity > 0 and bprice_check == "True":
            best_sell_id = sell_orderbook['Trader_ID'].iloc[0]
            best_sell_order_id = sell_orderbook['Order_ID'].iloc[0]
            best_sell_qty = sell_orderbook['Quantity'].iloc[0] 
            result = order_quantity - best_sell_qty
            price_diff = order_price - key_figs.best_ask
            written = False
            if result > 0:
                # this will clear the sell order as it can be executed against in its full entirety 
                transaction_append(timestamp, log_order_id, input_order_id, best_sell_order_id, best_sell_id, best_sell_qty, key_figs.best_ask, "Buy" ,input_order_id)
                
                # removing cleared sell order from orderbook, and resets index
                cleared_order = sell_orderbook.iloc[0]
                h_log_update(cleared_order, "passive_order_executed", timestamp)
                sell_orderbook = sell_orderbook.iloc[1:]
                sell_orderbook.reset_index(drop = True, inplace=True)
                bot = bot_crediting(key_figs.best_ask, best_sell_qty, input_order_id, best_sell_id, price_diff, "buy", bot)
                # updating order to be sent to orderbook_log with the new quantity 
                amend_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }
                amend_log_order = pd.Series(amend_log)
                h_log_amend (amend_log_order, result, "Buy", "Aggressive")
                order_quantity = result # reset order_quantity to new level
                
            elif result == 0:
                # this will clear the buy and sell order as it can be executed against in its full entirety 
                transaction_append(timestamp, log_order_id, input_order_id, best_sell_order_id, best_sell_id, best_sell_qty, key_figs.best_ask, "Buy" ,input_order_id)

                # removing cleared sell order from orderbook, and resets index
                cleared_order = sell_orderbook.iloc[0]
                h_log_update(cleared_order, "passive_order_executed", timestamp)
                sell_orderbook = sell_orderbook.iloc[1:]
                sell_orderbook.reset_index(drop = True, inplace=True)
                bot = bot_crediting(key_figs.best_ask, best_sell_qty, input_order_id, best_sell_id, price_diff, "buy", bot)
                # updating incoming sell order in orderbook_log
                clear_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }
                clear_log_order = pd.Series(clear_log)
                h_log_update(clear_log_order, "aggressive_order_executed", timestamp) # updating buy order to be executed
                order_quantity = 0
                break
            elif result < 0:
                # this clears the bid order against the sell order with left over quantity, writing the new quantity to the sell orderbook 
                transaction_append(timestamp, log_order_id, input_order_id, best_sell_order_id, best_sell_id, order_quantity, key_figs.best_ask, "Buy", input_order_id)
                bot = bot_crediting(key_figs.best_ask, order_quantity, input_order_id, best_sell_id, price_diff, "buy", bot)     
                empty_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }      
                empty_log_order = pd.Series(empty_log)
                h_log_update(empty_log_order, "aggressive_order_executed", timestamp) # updating buy order to be executed
                
                # updating the quantity of the partially-filled sell order
                new_sell_qty = best_sell_qty - order_quantity
                update_sellorder_index = sell_orderbook[(sell_orderbook["Order_ID"] == best_sell_order_id)].index
                amended_order = sell_orderbook.loc[update_sellorder_index]
                h_log_amend(amended_order, new_sell_qty, "Sell", "Passive") 
                sell_orderbook.loc[update_sellorder_index,"Quantity"] = new_sell_qty 
                order_quantity = 0
                break

            try: 
                # Pulling sell orderbook info
                key_figs.best_ask = sell_orderbook['Price'].iloc[0]
                best_sell_id = sell_orderbook['Trader_ID'].iloc[0]
                best_sell_order_id = sell_orderbook['Order_ID'].iloc[0]
                best_sell_qty = sell_orderbook['Quantity'].iloc[0]
                bprice_check = bid_price_check(order_price, key_figs.best_ask)   
                
            except:    
                if order_quantity > 0: # leftover quantity is written to the order book if sell orderbook is empty
                    written = True
                    orderbook_amend(log_order_id, input_order_id, timestamp, order_quantity, order_price, "Buy")
                break
        if order_quantity > 0 and (bprice_check == "False" and written == False):
            orderbook_amend(log_order_id, input_order_id, timestamp, order_quantity, order_price, "Buy")

    elif bid_ask_flag == "ask" and aprice_check == "True":   
        # appending to the orderbook log
        log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Timestamp" : timestamp, 
                "Quantity" : order_quantity, 
                "Price" : order_price, 
                "Side" : "Sell", 
                "Status" : "Open", 
                "Update_Timestamp" :timestamp,
                "Version" : 1
            }
        log_order = pd.Series(log)
        orderbook_log = pd.concat([orderbook_log, log_order.to_frame().T], ignore_index=True)

        # Running a while loop to execute the order against as many eligible orders 
        while order_quantity > 0 and aprice_check == "True":  
            best_buy_id = buy_orderbook['Trader_ID'].iloc[0]
            best_buy_order_id = buy_orderbook['Order_ID'].iloc[0]
            best_buy_qty = buy_orderbook['Quantity'].iloc[0]
            result = order_quantity - best_buy_qty        
            price_diff = key_figs.best_bid - order_price
            written=False
            if result > 0:
                # this will clear the buy order as it can be executed against in its full entirety 
                transaction_append(timestamp, best_buy_order_id, best_buy_id, log_order_id, input_order_id, best_buy_qty, key_figs.best_bid, "Sell", input_order_id)
    
                # removing cleared buy order from orderbook, and resets index
                cleared_order = buy_orderbook.iloc[0]
                h_log_update(cleared_order, "passive_order_executed", timestamp)
                buy_orderbook = buy_orderbook.iloc[1:]
                buy_orderbook.reset_index(drop = True, inplace=True)
                bot = bot_crediting(key_figs.best_bid, best_buy_qty, best_buy_id, input_order_id, price_diff, "sell", bot)  
                # updating order to be sent to orderbook_log with the new quantity 
                amend_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }
                amend_log_order = pd.Series(amend_log)
                h_log_amend (amend_log_order, result, "Sell", "Aggressive")
                order_quantity = result # reset order_quantity to new level
                result = order_quantity - best_buy_qty   
            elif result == 0:
                # this will clear the sell and buy order as it can be executed against in its full entirety 
                transaction_append(timestamp, best_buy_order_id, best_buy_id, log_order_id, input_order_id, best_buy_qty, key_figs.best_bid, "Sell", input_order_id)
    
                # removing cleared buy order from orderbook, and resets index
                cleared_order = buy_orderbook.iloc[0]
                h_log_update(cleared_order, "passive_order_executed", timestamp)
                buy_orderbook = buy_orderbook.iloc[1:]
                buy_orderbook.reset_index(drop = True, inplace=True)
                bot = bot_crediting(key_figs.best_bid, best_buy_qty, best_buy_id, input_order_id, price_diff, "sell", bot)  
                # updating incoming sell order in orderbook_log
                clear_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }
                clear_log_order = pd.Series(clear_log)
                h_log_update(clear_log_order, "aggressive_order_executed", timestamp) # updating sell order to be executed
                order_quantity = 0
                break
            elif result < 0:
                # this clears the ask order against the buy order with left over quantity, writing the new quantity to the buy orderbook 
                transaction_append(timestamp, best_buy_order_id, best_buy_id, log_order_id, input_order_id, order_quantity, key_figs.best_bid, "Sell", input_order_id)
                bot = bot_crediting(key_figs.best_bid, order_quantity, best_buy_id, input_order_id, price_diff, "sell", bot)
                empty_log = {
                "Order_ID" : log_order_id, 
                "Trader_ID" : input_order_id, 
                "Quantity" : order_quantity, 
                "Price" : order_price
                }
                empty_log_order = pd.Series(empty_log)
                h_log_update(empty_log_order, "aggressive_order_executed", timestamp) # updating sell order to be executed

                # updating the quantity of the partially-filled buy order
                new_buy_qty = best_buy_qty - order_quantity
                update_buyorder_index = buy_orderbook[(buy_orderbook["Order_ID"] == best_buy_order_id)].index
                amended_order = buy_orderbook.loc[update_buyorder_index]
                h_log_amend(amended_order, new_buy_qty, "Buy", "Passive")
                buy_orderbook.loc[update_buyorder_index,"Quantity"] = new_buy_qty
                order_quantity = 0
                break
            try:
                # Pulling buy orderbook info
                key_figs.best_bid = buy_orderbook['Price'].iloc[0]
                best_buy_id = buy_orderbook['Trader_ID'].iloc[0]
                best_buy_order_id = buy_orderbook['Order_ID'].iloc[0]
                best_buy_qty = buy_orderbook['Quantity'].iloc[0]
                aprice_check = ask_price_check(order_price, key_figs.best_bid)
            except:
                if order_quantity > 0: # leftover quantity is written to the order book
                    written = True
                    orderbook_amend(log_order_id, input_order_id, timestamp, order_quantity, order_price, "Sell")
                break
        if order_quantity > 0 and (bprice_check == "False" and written == False):
            orderbook_amend(log_order_id, input_order_id, timestamp, order_quantity, order_price, "Sell")
    return bot

def orderbook_vis (orderbook, side, html_flag):
    agg_orderbook = pd.DataFrame(columns=["Price", "Quantity"])
    agg_orderbook = orderbook.groupby("Price").agg({"Quantity" : 'sum'}).reset_index()
    agg_orderbook.columns = ['Price', 'Total Quantity']
    
    if html_flag == True:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        if side == "b":
            agg_orderbook.sort_values(by=['Price'], ascending=False, inplace=True) 
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
        vis = vis.get_html_string()
    elif html_flag == False:
        vis = PrettyTable()
        vis.field_names = ['Price', 'Total Quantity']
        for index, row in agg_orderbook.iterrows():
            vis.add_row([row['Price'], row['Total Quantity']])
    return vis

# T.1.1: cancelling orders away from priority
def cancel_faraway_orders (bot):
    global transaction_log, buy_orderbook, sell_orderbook
    key_figs = market_key_figures()
    if key_figs.key_figs_test == 'pass':
        if len(transaction_log) > 24:
            prev_price = transaction_log.iat[-25,-3]
        else:
            prev_price = transaction_log.iat[-1,-3]
        market_delta = prev_price - key_figs.market_price
        
        # checking for any orders that should be cancelled. If they are to be cancelled, the appropriate wealth/asset is credited back to the bot's account.    
        for index, order in buy_orderbook.iterrows():
            market_order_spread = key_figs.best_bid - order["Price"]
            timestamp = sim_to_real_time(dt.datetime.now())
            if bot.iloc[0] == order["Trader_ID"] and market_order_spread > 0.1: # 10 ticks away from priority
                bot["Wealth"] += order["Price"] * order["Quantity"]
                h_log_update(order, 'order_cancelled_price', timestamp)
                buy_orderbook.drop(index=index, inplace=True)
            elif bot.iloc[0] == order["Trader_ID"] and (market_order_spread <= 0.01 and market_delta > 0.02):           
                bot["Wealth"] += order["Price"] * order["Quantity"]
                h_log_update(order, 'order_cancelled_price', timestamp)
                buy_orderbook.drop(index=index, inplace=True)
                
        for index, order in sell_orderbook.iterrows():
            order_market_spread = order["Price"] - key_figs.best_ask
            timestamp = sim_to_real_time(dt.datetime.now())
            if bot.iloc[0] == order["Trader_ID"] and order_market_spread > 0.1:
                bot["Asset"] += order["Quantity"]
                h_log_update(order, 'order_cancelled_price', timestamp)
                sell_orderbook.drop(index=index, inplace=True)
            elif bot.iloc[0] == order["Trader_ID"] and (market_order_spread <= 0.01 and market_delta < -0.02):           
                bot["Asset"] += order["Quantity"]
                h_log_update(order, 'order_cancelled_price', timestamp)
                sell_orderbook.drop(index=index, inplace=True)
    return bot

# debiting function - takes order info, and debits the money or asset accordinlgy, from the bot
def bot_debiting(bot, input_orders):
    for index, order in input_orders.iterrows():
        qty = int(order["Quantity"])
        price = round(order["Price"],2)
        if order["Flag"] == "bid":
            order_value = price * qty
            bot.iloc[2] -= round(order_value,2) # wealth
        elif order["Flag"] == "ask":    
            bot.iloc[1] -= qty # assets
    return bot

def quantity_cap_check(qty):
    if qty > 1000:
        r_qty = 1000
    else:
        r_qty = qty
    return r_qty

# this function is to be called during the initial market startup i.e. key_figs_test = 'fail'. This does not run the matching logic, but rather fills the orderbook instead.
def bot_market_interact_initial (result, bot):
    global buy_orderbook, sell_orderbook, start_price
    key_figs = market_key_figures()
    timestamp = sim_to_real_time(dt.datetime.now())
    start_price = 5
    trader_id = bot["Trader_ID"]
    price_offest = round(abs(np.random.standard_normal()) * 4) * 0.01 
    max_buy_quantity = quantity_cap_check(bot["Wealth"] / start_price)
    max_sell_quantity = quantity_cap_check(bot["Asset"])
    df_input_orders = pd.DataFrame(columns=["Trader_ID", "Timestamp", "Quantity", "Price", "Flag"])

    if result == 'buy_order' or result == "buy_execute":
        order_price = round(start_price - price_offest,2)
        order_quantity = round(random.uniform(0.25, 0.45) * max_buy_quantity)
        order_val_check = order_price * order_quantity

        if order_val_check < bot["Wealth"]:
            input_order = pd.Series({"Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "bid"})
            df_input_orders = pd.concat([df_input_orders, input_order.to_frame().T], ignore_index=True)
            bot = bot_debiting(bot, df_input_orders)
            # This is a check to see if the orderbook_matching logic can be initaited i.e. if there are orders on both sides of the market
            if key_figs.key_figs_test == "o_semipass":
                bot = orderbook_matching(input_order, bot)
            else:
                buy_orderbook_append(trader_id, timestamp, order_quantity, order_price)
        
    elif result == 'sell_order' or result == "sell_execute":
        order_price = round(start_price + price_offest, 2)
        order_quantity = round(random.uniform(0.15, 0.35) * max_sell_quantity)
        
        if order_quantity < bot["Asset"]:      
            input_order = pd.Series({"Trader_ID" : trader_id, "Timestamp" : timestamp, "Quantity" : order_quantity, "Price" : order_price, "Flag" : "ask"})     
            df_input_orders = pd.concat([df_input_orders, input_order.to_frame().T], ignore_index=True)
            bot = bot_debiting(bot, df_input_orders)          
            # This is a check to see if the orderbook_matching logic can be initaited i.e. if there are orders on both sides of the market
            if key_figs.key_figs_test == "o_semipass":
                bot = orderbook_matching(input_order, bot)  
            else:
                sell_orderbook_append(trader_id, timestamp, order_quantity, order_price)
    return bot

# Writing results to excel file
def file_write(file_name): 
    global df_participants, transaction_log, buy_orderbook, sell_orderbook, orderbook_log
    final_output_path = os.path.join(output_path, file_name)
    df_participants.to_excel(final_output_path)
    with pd.ExcelWriter(final_output_path) as writer:
        df_participants.to_excel(writer, sheet_name='Participants')
        transaction_log.to_excel(writer,sheet_name='Transaction_Log')
        buy_orderbook.to_excel(writer, sheet_name='Buy_Orderbook')
        sell_orderbook.to_excel(writer, sheet_name='Sell_Orderbook')
        orderbook_log.to_excel(writer,sheet_name='Orderbook_Log')
#####################################################################
############ End of Market Functions ################################
#####################################################################
############ Beginning of Widget Setup ##############################
#####################################################################
simulation_state = "none"
simulation_thread = None
# Creating widgets for the front end
simulation_button = widgets.ToggleButtons(
    options=["Start", "Pause", "Resume", "End"],
    disabled=False,
    value=None
)
bot_num_widget = widgets.IntText(
    value=100,
    description='Number of computer participants:',
    disabled=False,
    style={'description_width': 'initial'}
)
liquidity_setting_widget = widgets.Dropdown(
    options=['High', 'Medium', 'Low'],
    value = 'High',
    description = 'Select Liquidity:',
    disabled = False,
    style={'description_width': 'initial'}
)
timer_input = widgets.Checkbox(
    value=False,
    description='Select to specify simulation run time.',
    disabled=False,
    indent=False
)
sim_time_widget = widgets.IntText(
    value=180,
    description='Simulation run time (seconds):',
    disabled=True,
    style={'description_width': 'initial'}
)
market_data_check = widgets.Checkbox(
    value=False,
    description='Select to fabricate market data.',
    disabled=True,
    indent=False
)
market_data_length = widgets.Dropdown(
    options=['24 Hours', '8.5 Hours (LSE)', '6.5 Hours (NYSE)', '6 Hours (TSE)', '1 Hour'],
    value='24 Hours',
    description='Select emulation:',
    disabled=True,
    style={'description_width': 'initial'}
)
file_input = widgets.Checkbox(
    value=False,
    description='Select for output of results to file.',
    disabled=False,
    indent=False
)
file_name_input = widgets.Text(
    value="",
    desciprion='Enter file name (saved to comms folder) e.g. Output.xlsx:',
    disabled=True,
    style={'description_width': 'initial'}
)
def timer_check_change(change):
    sim_time_widget.disabled = not change['new']
    market_data_check.disabled = not change['new']
timer_input.observe(timer_check_change, names='value')

def market_data_change(change):
    market_data_length.disabled = not change['new']
market_data_check.observe(market_data_change, names='value')

def file_check_change(change):
    file_name_input.disabled = not change['new']
file_input.observe(file_check_change, names='value')

grouped_widgets = [widgets.VBox([bot_num_widget, 
                                 liquidity_setting_widget, 
                                 timer_input, 
                                 sim_time_widget, 
                                 market_data_check, 
                                 market_data_length, 
                                 file_input, 
                                 file_name_input])]
sim_setup = widgets.Accordion(
    children=(grouped_widgets),
    titles=(['Simulation Setup'])
)

market_widget = widgets.HTML()
b_orderbook_widget = widgets.HTML()
s_orderbook_widget = widgets.HTML()
recent_transactions_widget = widgets.HTML()
chart_widget = widgets.Image(format='png')
sentiment_widget = widgets.HTML()
center_grid = widgets.GridspecLayout(1,3)
user_grid = widgets.GridspecLayout(3,5)
user_profile_widget = widgets.HTML()
user_positions_grid = widgets.GridspecLayout(4,3)
quantity_grid = widgets.GridspecLayout(2,1)
user_cancel_grid = widgets.GridspecLayout(1,2)

# Creating widgets to cancel user orders 
user_cancel_header_html = widgets.HTML(
    value = """
    <html>
        <body>
            <br>
            <i align="center">
                Enter Order ID to cancel:
            </i>
        </body>
    </html>
    """
)
user_cancel = widgets.IntText(
    value=1,
    layout=widgets.Layout(width='50px', height='50px')
)
user_cancel_b = widgets.Button(
    description='Cancel Order',
    button_style='danger'
)
user_cancel_select = widgets.Dropdown(
    options=['Buy', 'Sell'],
    disabled = False,
    layout=widgets.Layout(width='100px', height='50px')
)
user_cancel_html = widgets.HTML()

# Widgets to display user orders
user_borderbook_html = widgets.HTML()
user_sorderbook_html = widgets.HTML()
user_buy_orders_widget = widgets.HTML()
user_sell_orders_widget = widgets.HTML()

# Define widgets for user interaction
order_side_widget = widgets.ToggleButtons(
    options=['Buy', 'Sell'],
    description='Order Side:',
    disabled=False,
    button_style = 'info'
)
order_type_widget = widgets.ToggleButtons(
    options=['Order', 'Execute'],
    description='Order Type:',
    disabled=False,
    button_style = 'primary'
)
price_widget = widgets.FloatText(
    value=0, 
    description='Price:',
    step=0.01,
    disabled=False,
    layout=widgets.Layout(width='250px')
)
quantity_widget = widgets.IntSlider(
    value=1,
    min=1,
    max=1,
    step=1,
    description='Quantity:',
    continuous_update=True
)
quantity_int = widgets.IntText(
    value=1,
    min=1,
    step=1,
    layout=widgets.Layout(width='100px', height='50px', margin='auto')
)
submit_button = widgets.Button(
    description='Place Order',
    disabled=False,
    button_style='success', 
    tooltip='Click to place order',
    icon='check'
)
widgets.link((quantity_widget, 'value'), (quantity_int, 'value'))
user_buy_total = widgets.HTML()
user_sell_total = widgets.HTML()
# Ordering widgets into a grid
quantity_grid[0,0] = quantity_widget
quantity_grid[1,0] = quantity_int
center_grid[0,0] = recent_transactions_widget
center_grid[0,1] = chart_widget
center_grid[0,2] = sentiment_widget
center_grid[:,0].layout.width = '150px'
center_grid[:,1].layout.width = '1000px'
center_grid[:,2].layout.width = '250px'
# Ordering cancel widgets into smaller grid
user_cancel_grid[0,0] = user_cancel_select
user_cancel_grid[0,1] = user_cancel

user_positions_grid[0,0] = user_borderbook_html
user_positions_grid[0,1] = user_sorderbook_html
user_positions_grid[0,2] = user_cancel_header_html
user_positions_grid[1:3,0] = user_buy_orders_widget
user_positions_grid[1:3,1] = user_sell_orders_widget
user_positions_grid[3,0] = user_buy_total
user_positions_grid[3,1] = user_sell_total
user_positions_grid[1,2] = user_cancel_grid
user_positions_grid[2,2] = user_cancel_b
user_positions_grid[3,2] = user_cancel_html
user_cancel_b.layout.width = '90%'

user_grid[:,0] = user_profile_widget
user_grid[0,1] = order_side_widget
user_grid[1,1] = order_type_widget
user_grid[0,2] = price_widget
user_grid[1,2] = quantity_grid
user_grid[2,1:3] = submit_button
user_grid[:,3:] = user_positions_grid
submit_button.layout.width = '100%'
user_grid[:, 0].layout.width = '300px'
user_grid.layout.grid_template_columns = '300px auto auto auto auto'

def start_simulation():
    global simulation_state, simulation_thread, pause_duration, stop_simulation_flag
    if stop_simulation_flag.is_set():
        print("Sim has been ended. Restarting the thread is not allowed.")
        return
    if simulation_state == "none" or simulation_state =="stopped":
        pause_duration = 0
        simulation_state = "running"
        simulation_thread = threading.Thread(target=simulation, daemon=True).start()
        print("Simulation started.") 

# function to update front-end widgets
def update_frontend():
    global transaction_log, buy_orderbook, sell_orderbook, df_participants, candlestick_df, front_end, orderbook_log
    global IB_market_state, IB_state_countdown, RI_market_state, RI_state_countdown, PI_market_state, PI_state_countdown, candle_length
    key_figs = market_key_figures()
    if key_figs.buy_orderbook_test == "b_fail":
        key_figs.best_bid = "Not Available"

    if key_figs.sell_orderbook_test == "s_fail":
        key_figs.best_ask = "Not Available"

    if len(orderbook_log) < 1:
        recent_order_id = 1
    else:
        recent_order_id = int(orderbook_log.iloc[-1,0])

    # Updating the front end html strings with market figures and market sentiments
    html_string = """
    <html>
        <body>
            <h1 align="center">Market Price: {market_price}</h1>
            <h2 style="text-align:left;">
            Best Bid: {best_bid}
            <span style="float:right;">
                Best Ask: {best_ask}
            </span>
            </h2>
            <p align="center" style="color:SlateBlue">
                Open Price: {open_price}&emsp;&emsp;&ensp;Absolute Delta: {abs_price_mvmt}&emsp;&emsp;Percentage Delta: {per_price_mvmt}%
            </p>
            <p align="center" style="color:SlateBlue">
                Max Price: {price_max}&emsp;&emsp;&emsp;&emsp;Min Price: {price_min}&emsp;&emsp;&emsp;&emsp;Price Range: {price_range}
            </p>
            <h3 style="text-align:left">
            <i>Buy Orderbook&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;&emsp;Recent Transactions</i>
            <span style="float:right">
                <i>Sell Orderbook</i>
            </span>
            </h3>      
        </body>
    </html>
    """.format(market_price=key_figs.market_price, 
    best_bid=key_figs.best_bid, 
    best_ask=key_figs.best_ask, 
    open_price=key_figs.open_price, 
    abs_price_mvmt=key_figs.abs_price_mvmt,
    per_price_mvmt=key_figs.per_price_mvmt,
    price_max=key_figs.price_max,
    price_min=key_figs.price_min,
    price_range=key_figs.price_range)

    sentiment_string="""
    <html>
        <body>
            <h3 align="center">Current Market Sentiments</h3>
            <p style="text-align:left;">Investment banker market sentiment is: </p> 
            <b>&emsp;{IB_sentiment}</b>
            <i>, with {IB_seconds} seconds remaining.</i>
            <p style="text-align:left;">Retail investor market sentiment is: </p> 
            <b>&emsp;{RI_sentiment}</b>
            <i>, with {RI_seconds} seconds remaining.</i>
            <p style="text-align:left;">Private investor market sentiment is: </p> 
            <b>&emsp;{PI_sentiment}</b>
            <i>, with {PI_seconds} seconds remaining.</i>
            <br>
            <br>
            <i> There have been {recent_order_id} orders placed. </i>
            <br>
            <br>
            <i> Current displayed candles represent {candle_length}. </i>
        </body>
    </html>
    """.format(IB_sentiment=IB_market_state,
    IB_seconds=IB_state_countdown,
    RI_sentiment=RI_market_state,
    RI_seconds=RI_state_countdown,
    PI_sentiment=PI_market_state,
    PI_seconds=PI_state_countdown,
    recent_order_id=recent_order_id,
    candle_length=candle_length)

    # Obtaining a list of the 10 most recent transactions to display
    if len(transaction_log) < 10:
        recent_transactions = transaction_log[["Timestamp", "Price", "Quantity", "Aggressor"]]
        recent_transactions.sort_values(by=["Timestamp", "Quantity"], ascending=[False, False], inplace=True)
    else:
        recent_transactions = transaction_log[["Timestamp", "Price", "Quantity", "Aggressor"]].tail(10)
        recent_transactions.sort_values(by=["Timestamp", "Quantity"], ascending=[False, False], inplace=True)
    
    rt_vis = PrettyTable()
    rt_vis.field_names = ['Time','Price','Quantity','Aggressor']
    for index, row in recent_transactions.iterrows():
        row["Timestamp"] = row["Timestamp"].time()
        rt_vis.add_row([row['Timestamp'], row['Price'], row['Quantity'], row['Aggressor']])
    recent_transactions_widget.value  = rt_vis.get_html_string()

    # Creating the front end graphs and orderbook visualisations 
    chart_colors = mpf.make_marketcolors(
        up='green',
        down='red'
    )
    chart_style = mpf.make_mpf_style(
        marketcolors = chart_colors,
        gridcolor='lightgray',        
        gridstyle='--'
    )
    front_end = candlestick_df.copy()
    # check if timestamp column exsists, and sets to index if it does not
    if "timestamp" in front_end.columns:
        front_end.set_index('timestamp', inplace=True)

    cols = ["open", "high", "low", "close"]
    front_end[cols] = front_end[cols].astype(float)
    front_end["volume"] = front_end["volume"].astype(int)
    fig, ax = mpf.plot(front_end, type='candle', style=chart_style, volume=True, returnfig=True)   
    buf = io.BytesIO()
    fig.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    chart_widget.value = buf.getvalue()
    sentiment_widget.value = sentiment_string
    market_widget.value = html_string
    b_orderbook_widget.value = orderbook_vis(buy_orderbook, "b", html_flag=True)
    s_orderbook_widget.value = orderbook_vis(sell_orderbook, "s", html_flag=True)

# function to update user's profile widget
def update_user():
    global df_participants, user_positions_grid, user_profile_widget, user_buy_orderbook, user_sell_orderbook, buy_orderbook, sell_orderbook
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_asset = round(user_profile.iloc[0]['Asset'],2)
    user_wealth = round(user_profile.iloc[0]['Wealth'],2)  
    user_id = user_profile.iloc[0]['Trader_ID']  
    user_html = """
    <html>
        <body>
            <h2> User profile </h2> 
            <i>&ensp; Last updated on: </i>
            <br>
            <i>&emsp; {timestamp} </i>
            <br>
            <br>
            <b align="center"> &emsp;&emsp;Current Assets: {user_asset}</b>  
            <br>
            <b align="center"> &emsp;&emsp;Current Wealth: {user_wealth}</b> 
            <br>
            <sub align="center"> &emsp;&emsp;Starting Assets: 100000</sub>
            <br>
            <sub align="center"> &emsp;&emsp;Starting Wealth: 100000</sub> 
        </body>
    </html>
    """.format(timestamp=dt.datetime.now(),user_asset=user_asset, user_wealth=user_wealth)
    user_profile_widget.value = user_html

    # Finding user orders, setting datatypes and resetting index
    user_buy_orderbook = buy_orderbook[buy_orderbook["Trader_ID"] == user_id]
    user_sell_orderbook = sell_orderbook[sell_orderbook["Trader_ID"] == user_id]   
    user_buy_orderbook.reset_index(inplace=True)
    user_sell_orderbook.reset_index(inplace=True)
    user_buy_orderbook.drop(columns='Trader_ID')
    user_sell_orderbook.drop(columns='Trader_ID')

    if len(user_buy_orderbook) > 0:
        buy_order_id_vals = list(range(1, len(user_buy_orderbook) + 1)) # creating a list of OrderIDs based on how many buy orders the user has, starting at 1
        user_buy_orderbook.insert(0, "OrderID", buy_order_id_vals)
        user_buy_orderbook.loc[:, "Timestamp"] = user_buy_orderbook['Timestamp'].dt.strftime('%H:%M:%S') 

    if len(user_sell_orderbook) > 0:
        sell_order_id_vals = list(range(1, len(user_sell_orderbook) + 1 )) # creating a list of OrderIDs based on how many sell orders the user has, starting at 1
        user_sell_orderbook.insert(0, "OrderID", sell_order_id_vals)
        user_sell_orderbook.loc[:, "Timestamp"] = user_sell_orderbook['Timestamp'].dt.strftime('%H:%M:%S')

    # Creating the two tables to display user orders and calculating total aggregated open positions
    buy_order_sum = 0
    buy_order_val = 0
    sell_order_sum = 0
    sell_order_val = 0
    ub_vis = PrettyTable()
    ub_vis.field_names = ['OrderID','Time','Price', 'Quantity']
    for index, row in user_buy_orderbook.iterrows():
        buy_order_sum += row['Quantity']
        buy_order_val += row['Quantity'] * row['Price']
        ub_vis.add_row([row["OrderID"], row['Timestamp'], row['Price'], row['Quantity']])
    user_buy_orders_widget.value  = ub_vis.get_html_string()

    us_vis = PrettyTable()
    us_vis.field_names = ['OrderID','Time','Price', 'Quantity']
    for index, row in user_sell_orderbook.iterrows():
        sell_order_sum += row['Quantity']
        sell_order_val += row['Quantity'] * row['Price']
        us_vis.add_row([row["OrderID"], row['Timestamp'], row['Price'], row['Quantity']])
    user_sell_orders_widget.value  = us_vis.get_html_string()

    buy_order_val = round(buy_order_val,2)
    sell_order_val = round(sell_order_val,2)

    user_borderbook_html.value="""
    <html>
        <body>
            <h3 style="text-align:left; color:MediumSeaGreen">
            <i>Open User Buy Positions</i>
            </h3>
        </body>
    </html>
    """
    user_sorderbook_html.value="""
    <html>
        <body>
            <h3 style="text-align:left; color:Tomato">
            <i>Open User Sell Positions</i>
            </h3>
        </body>
    </html>
    """
    user_buy_total.value="""
    <html>
        <body>
            <p style="text-align:center">
            Total Buy Open Order Quantity: {buy_order_sum}; 
            <br>
            with value: {buy_order_val}. 
            </p>
        </body>
    </html>
    """.format(buy_order_sum = buy_order_sum, buy_order_val = buy_order_val)
    user_sell_total.value= """ 
    <html>
        <body>
            <p style="text-align:center">
            Total Sell Open Order Quantity: {sell_order_sum}; 
            <br>
            with value: {sell_order_val}. 
            </p>
        </body>
    </html>
    """.format(sell_order_sum = sell_order_sum, sell_order_val = sell_order_val)

# triggered when pause putton is clicked 
def pause_simulation():
    global simulation_state, df_participants, buy_orderbook, sell_orderbook, user_positions_grid, user_profile_widget, pause_starttime, user_buy_orderbook, user_sell_orderbook

    if simulation_state =="running":
        simulation_state = "paused"
        print("Simulation paused.")
    pause_starttime = time.time()

    key_figs = market_key_figures()
    update_frontend()
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_asset = user_profile.iloc[0]['Asset']
    user_wealth = user_profile.iloc[0]['Wealth']
    price_widget.value = key_figs.market_price

    # Function to update the quantity slider's range
    def update_qty_slider(*args):
        price = price_widget.value
        o_type = order_type_widget.value
        o_side = order_side_widget.value

        if o_type == "Order" and o_side == "Buy":
            max_quantity = int(user_wealth / price) if price > 0 else 0.1
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
        elif o_type == "Execute" and o_side == "Buy":
            max_quantity = int(user_wealth / key_figs.best_ask) # best ask
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
        elif (o_type == "Order" or o_type == "Execute") and o_side == "Sell":
            max_quantity = int(user_asset)
            quantity_widget.max = max_quantity
            quantity_int.max = max_quantity
            
    def update_price_app(*args):
        o_type = order_type_widget.value
        o_side = order_side_widget.value
        if o_type == "Execute" and o_side == "Buy":
            price_widget.value = key_figs.best_ask # best ask
            price_widget.disabled = True
        elif o_type == "Execute" and o_side == "Sell":
            price_widget.value = key_figs.best_bid # best bid
            price_widget.disabled = True
        else: 
            price_widget.disabled = False

    update_user()

    update_price_app()
    order_side_widget.observe(update_price_app, 'value')
    order_type_widget.observe(update_price_app, 'value')

    update_qty_slider()
    price_widget.observe(update_qty_slider, 'value')
    
# Function to handle button click and place order
def on_submit_button_clicked(b):
    global df_participants
    key_figs = market_key_figures()
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_id = user_profile.iloc[0]['Trader_ID']
    order_side = order_side_widget.value
    order_type = order_type_widget.value
    price = round(price_widget.value, 2)
    quantity = round(quantity_widget.value,2)
    timestamp = sim_to_real_time(dt.datetime.now())

    # Logic to place the order into the market
    if order_side == "Buy" and order_type == "Order":
        input_order = pd.Series({"Trader_ID" : user_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price, "Flag" : "bid"})

    elif order_side == "Sell" and order_type == "Order": 
        input_order = pd.Series({"Trader_ID" : user_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : price, "Flag" : "ask"})
        
    elif order_side == "Buy" and order_type == "Execute":
        input_order = pd.Series({"Trader_ID" : user_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : key_figs.best_ask, "Flag" : "bid"})

    elif order_side =="Sell" and order_type == "Execute":
        input_order = pd.Series({"Trader_ID" : user_id, "Timestamp" : timestamp, "Quantity" : quantity, "Price" : key_figs.best_bid, "Flag" : "ask"})
        
    input_orders = pd.DataFrame(columns=["Timestamp", "Trader_ID", "Quantity", "Price", "Flag"])
    input_orders = pd.concat([input_orders, input_order.to_frame().T], ignore_index=True)
    profile = df_participants.loc[df_participants["Profile"] == "User"].squeeze()
    profile = bot_debiting(profile, input_orders)
    profile = orderbook_matching(input_order, profile)
    df_profile = profile.to_frame().T
    df_profile = df_profile.astype(df_participants.dtypes.to_dict())
    df_profile = df_profile.reindex(columns=df_participants.columns)
    df_participants.loc[df_participants["Profile"] == "User"] = df_profile
    update_user()
    update_frontend()

def on_cancel_button_clicked(b):
    global df_participants, sell_orderbook, buy_orderbook, user_sell_orderbook, user_buy_orderbook
    order_id = user_cancel.value
    order_side = user_cancel_select.value
    user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
    user_id = user_profile.iloc[0]['Trader_ID']
    if order_side == "Buy" and (user_buy_orderbook.empty == True or order_id not in user_buy_orderbook["OrderID"].values):
        user_cancel_html.value="""
        <html>
            <body>
                <p style="text-align:left; color:Tomato">
                    Buy order {OrderID} does not exist.
                <p>
            </body>
        </html>
        """.format(OrderID=order_id)
    elif order_side == "Sell" and (user_sell_orderbook.empty == True or order_id not in user_sell_orderbook["OrderID"].values):
        user_cancel_html.value="""
        <html>
            <body>
                <p style="text-align:left; color:Tomato">
                    Sell order {OrderID} does not exist.
                <p>
            </body>
        </html>
        """.format(OrderID=order_id)
    elif  order_side == "Buy" and order_id in user_buy_orderbook["OrderID"].values:
        # take the price, and quantity of the order
        cancel_order_price = user_buy_orderbook.loc[user_buy_orderbook['OrderID'] == order_id, 'Price'].values[0]
        cancel_order_qty = user_buy_orderbook.loc[user_buy_orderbook['OrderID'] == order_id, 'Quantity'].values[0]
        # boolean condition to find the order to cancel
        cancel_order = (
            (buy_orderbook["Trader_ID"] == user_id) &
            (buy_orderbook["Price"] == cancel_order_price) &
            (buy_orderbook["Quantity"] == cancel_order_qty)
        )
        cancel_order_val = cancel_order_price * cancel_order_qty
        df_participants.loc[df_participants["Trader_ID"] == user_id, 'Wealth'] += cancel_order_val
        
        index_to_drop = buy_orderbook[cancel_order].index[0]
        cancel_order_time = buy_orderbook.loc[index_to_drop, "Timestamp"]
        cancel_order_id = buy_orderbook.loc[index_to_drop, "Order_ID"]
        input_order = pd.Series({"Order_ID" : cancel_order_id, "Trader_ID" : user_id, "Timestamp" : cancel_order_time, "Quantity" : cancel_order_qty, "Price" : cancel_order_price})
        cancel_time = sim_to_real_time(dt.datetime.now())
        h_log_update(input_order, "order_cancelled", cancel_time)
        buy_orderbook.drop(index_to_drop, inplace=True)
        
        user_cancel_html.value="""
        <html>
            <body>
                <p style="text-align:left; color:Tomato">
                    Buy order {OrderID} at {price} with {qty} quantity 
                    <br> has been cancelled.
                <p>
            </body>
        </html>
        """.format(OrderID=order_id,
        price=cancel_order_price,
        qty=cancel_order_qty)
        update_user()
        update_frontend()
    elif order_side == "Sell" and order_id in user_sell_orderbook["OrderID"].values:
        # take the price, and quantity of the order
        cancel_order_price = user_sell_orderbook.loc[user_sell_orderbook['OrderID'] == order_id, 'Price'].values[0]
        cancel_order_qty = user_sell_orderbook.loc[user_sell_orderbook['OrderID'] == order_id, 'Quantity'].values[0]
        # boolean condition to find the order to cancel
        cancel_order = (
            (sell_orderbook["Trader_ID"] == user_id) &
            (sell_orderbook["Price"] == cancel_order_price) &
            (sell_orderbook["Quantity"] == cancel_order_qty)
        )
        df_participants.loc[df_participants["Trader_ID"] == user_id, 'Asset'] += cancel_order_qty
        
        index_to_drop = sell_orderbook[cancel_order].index[0]
        cancel_order_time = sell_orderbook.loc[index_to_drop, "Timestamp"]
        cancel_order_id = sell_orderbook.loc[index_to_drop, "Order_ID"]
        input_order = pd.Series({"Order_ID" : cancel_order_id, "Trader_ID" : user_id, "Timestamp" : cancel_order_time, "Quantity" : cancel_order_qty, "Price" : cancel_order_price})
        cancel_time = sim_to_real_time(dt.datetime.now())
        h_log_update(input_order, "order_cancelled", cancel_time)
        sell_orderbook.drop(index_to_drop, inplace=True)
        user_cancel_html.value="""
        <html>
            <body>
                <p style="text-align:left; color:Tomato">
                    Sell order {OrderID} at {price} with {qty} quantity 
                    <br> has been cancelled.
                <p>
            </body>
        </html>
        """.format(OrderID=order_id,
        price=cancel_order_price,
        qty=cancel_order_qty)
        update_user()
        update_frontend()

# Attach the callback function to the button
submit_button.on_click(on_submit_button_clicked)
user_cancel_b.on_click(on_cancel_button_clicked)

def resume_simulation():
    global simulation_state, pause_starttime, pause_duration
    if simulation_state == "paused":
        simulation_state = "running"
        print("Simulaiton resumed.")
    resume_time = time.time()
    pause_duration += resume_time - pause_starttime

stop_simulation_flag = threading.Event()
pause_simulation_flag = threading.Event()
inverse_sim_run_flag = threading.Event() # set to false when running, and set to true when iteration is finished

def stop_simulation():
    global simulation_state
    file_write_bool = file_input.value
    file_name = file_name_input.value + ".xlsx"
    simulation_state = "stopped"
    if simulation_thread is not None:
        simulation_thread.join()
    if file_write_bool == True:
            file_write(file_name)
            print("File output has been written successfully.")   
    print("Simulation stopped.")
    update_frontend()
    update_user()
#####################################################################
############################### End of Widget Setup #################
#####################################################################
############################### Market Simulation ###################
#####################################################################

def simulation():
    global buy_orderbook, sell_orderbook, transaction_log, df_participants, simulation_state, pause_duration, start_time, sim_duration, timed_sim, time_elapsed, orderbook_log, candlestick_df, df_mm
    global IB_market_state, IB_state_countdown, RI_market_state, RI_state_countdown, PI_market_state, PI_state_countdown, current_window_start, time_scalar, start_time, risk_df, df_decisions
    participant_num = bot_num_widget.value
    sim_duration = sim_time_widget.value
    timed_sim = timer_input.value
    emulation_flag = market_data_check.value
    emulation_length = market_data_length.value
    liquidity_setting = liquidity_setting_widget.value

    buy_orderbook = pd.DataFrame(columns=["Order_ID", "Trader_ID", "Timestamp", "Quantity", "Price"])
    sell_orderbook = pd.DataFrame(columns=["Order_ID", "Trader_ID", "Timestamp", "Quantity", "Price"])
    transaction_log = pd.DataFrame(columns=["Timestamp", "Buy_Side_Order_ID", "Buy_Side_Trader_ID", "Sell_Side_Order_ID", "Sell_Side_Trader_ID", "Quantity", "Price", "Aggressor", "Aggressor_ID"])   
    orderbook_log = pd.DataFrame(columns=["Order_ID", "Trader_ID", "Timestamp", "Quantity", "Price", "Side", "Status", "Update_Timestamp", "Version"])
    candlestick_df = pd.DataFrame(columns=['open', 'high', 'low', 'close', 'volume'], index=pd.Index([], name='timestamp'))
    risk_df = pd.DataFrame(columns=["Trader_ID", "Old Risk", "New Risk", "PnL", "Exposure" , "Volatility"])
    df_decisions = pd.DataFrame(columns=["Trader_ID", "Profile", "Risk", "Result", "T_Open", "Tree1", "Tree2", "Tree3", "Tree4", "Tree5", "Tree6", "Transactions", "Prio", "Emotion", "Liquidity"])
    start_time = time.time()
    current_window_start = dt.datetime.now()

    # Calcualting by how much time should be scaled 
    def time_scalar_calc(run_time, emulation_length):
        if emulation_length == "24 Hours":
            time_scalar = 86400 / run_time # 24 hrs in seconds
        elif emulation_length == "8.5 Hours (LSE)":
            time_scalar = 30600 / run_time # 8.5 hrs in seconds
        elif emulation_length == "6.5 Hours (NYSE)":
            time_scalar = 23400 / run_time # 6.5 hrs in seconds
        elif emulation_length == "6 Hours (TSE)":
            time_scalar = 21600 / run_time # 6 hrs in seconds
        elif emulation_length == "1 Hour":
            time_scalar = 3600 / run_time # 1 hr in seconds 
        return time_scalar
    if emulation_flag == True:
        time_scalar = time_scalar_calc(sim_duration, emulation_length)
    elif emulation_flag == False:
        time_scalar = 1

    # Setup for market sentiment module
    IB_states = ["bull", "neutral", "bear"]
    p_states = [0.15, 0.7, 0.15]
    IB_market_state_duration = 0
    RI_market_state_duration = 0
    PI_market_state_duration = 0
    IB_market_state_start = time.time()
    RI_market_state_start = IB_market_state_start
    PI_market_state_start = IB_market_state_start
    IB_market_state = None
    RI_market_state = None
    PI_market_state = None
    grace_period = start_time + 10

    def IB_market_sentiment():
        IB_market_state = np.random.choice(a=IB_states, p=p_states)
        market_state_duration = random.randint(1, 90)
        market_state_start = time.time()
        return IB_market_state, market_state_start, market_state_duration

    def RI_market_sentiment(market_state):
        market_state_duration = IB_market_state_duration + random.randint(1, 60)
        market_state_start = time.time()
        if market_state == "bull":
            hyper_bull_states = ["bull", "h_bull"]
            h_bull_p = [0.6, 0.4]
            RI_market_state = np.random.choice(a=hyper_bull_states, p=h_bull_p)
        elif market_state == "bear":
            hyper_bear_states = ["bear", "h_bear"]
            h_bear_p = [0.6, 0.4]
            RI_market_state = np.random.choice(a=hyper_bear_states, p=h_bear_p)
        elif market_state == "neutral":
            RI_market_state = "neutral"
        return RI_market_state, market_state_start, market_state_duration

    def PI_market_sentiment(market_state):
        market_state_duration = IB_market_state_duration + random.randint(1, 30)
        market_state_start = time.time()
        if market_state == "bull":
            hyper_bull_states = ["bull", "h_bull"]
            h_bull_p = [0.8, 0.2]
            PI_market_state = np.random.choice(a=hyper_bull_states, p=h_bull_p)
        elif market_state == "bear":
            hyper_bear_states = ["bear", "h_bear"]
            h_bear_p = [0.8, 0.2]
            PI_market_state = np.random.choice(a=hyper_bear_states, p=h_bear_p)
        elif market_state == "neutral":
            PI_market_state = "neutral"
        return PI_market_state, market_state_start, market_state_duration

    # resets the bot delay timer, using the inverse of the activity float (larger the number, more often they will interact), and a random integer to produce variation
    def timer_reset (bot):
        delay = round((1/bot["Activity"]) * abs(np.random.randint(5,50)))
        bot["Delay"] = delay + 1
        return bot

    # participant and user setup
    df_participants = ps.participant_creation(participant_num, liquidity_setting)
    df_participants = ps.user_creation(df_participants, participant_num)
    df_participants = df_participants.astype({"Trader_ID":int, "Asset":float, "Wealth":float, "Risk":float, "Activity":float, "Delay":int, "Profile":str, "PreAsset":float, "PreWealth":float})

    # Maing program run loop
    while simulation_state != "stopped" and not stop_simulation_flag.is_set():
        inverse_sim_run_flag.clear()
        time_elapsed = time.time() - start_time
        if time_elapsed > sim_duration and timed_sim == True:
            print("Sim duration has expired.")
            simulation_state = "end"
            stop_simulation_flag.set()
            pause_simulation_flag.set()
            stop_simulation()
            break
        if simulation_state=="paused" and pause_simulation_flag.is_set():
            pause_simulation_flag.wait()
        elif simulation_state == "running":   
            # Timers for the visualiation, and sentiment modules. When timers run out, they execute the function i.e. updating vis or marjet sentiment 
            IB_timer = time.time() - IB_market_state_start
            RI_timer = time.time() - RI_market_state_start
            PI_timer = time.time() - PI_market_state_start
            IB_state_countdown = round(IB_market_state_duration - IB_timer)
            RI_state_countdown = round(RI_market_state_duration - RI_timer)
            PI_state_countdown = round(PI_market_state_duration - PI_timer) 
            if IB_state_countdown < 0 and time.time() > grace_period:
                IB_market_state, IB_market_state_start, IB_market_state_duration = IB_market_sentiment()
            if RI_state_countdown < 0 and time.time() > grace_period:
                RI_market_state, RI_market_state_start, RI_market_state_duration = RI_market_sentiment(IB_market_state)
            if PI_state_countdown < 0 and time.time() > grace_period:
                PI_market_state, PI_market_state_start, PI_market_state_duration = PI_market_sentiment(IB_market_state)
  
            # If an orderbook is empty, all other trading is stopped, and market maker will fill the liquidity gap    
            key_figs = market_key_figures()    
            if key_figs.key_figs_test == "t_semipass":
                df_mm = ps.mm_liquidity_fill(df_participants)
                for index, mm in df_mm.iterrows():
                    mm, input_orders= bo.liquidity_creator(mm, key_figs, timestamp)
                    mm = bot_debiting(mm, input_orders)
                    for index, order in input_orders.iterrows():
                        mm = orderbook_matching(order, mm)
                    merged_df = df_participants.merge(df_mm, on="Trader_ID", how='left', suffixes=('_old', '_new'))
                    merged_df['Asset'] = merged_df['Asset_new'].fillna(merged_df['Asset_old']).infer_objects(copy=False)
                    merged_df['Wealth'] = merged_df['Wealth_new'].fillna(merged_df['Wealth_old']).infer_objects(copy=False)
                    merged_df['Delay'] = merged_df['Delay_new'].fillna(merged_df['Delay_old']).infer_objects(copy=False)
                    merged_df.drop(['Asset_old', 'Wealth_old', 'Delay_old', 'Asset_new', 'Wealth_new', 'Delay_new'], axis=1, inplace=True)

                # ensuring compatible dtypes, then updating
                merged_df['Trader_ID'] = merged_df['Trader_ID'].astype(int)
                df_participants.update(merged_df)
                key_figs = market_key_figures()

            elif key_figs.key_figs_test != "t_semipass":
                df_available = ps.iteration_start(df_participants)
                # running the bot decisions for the available bots
                for index, bot in df_available.iterrows():
                    key_figs = market_key_figures()
                    tree1_cancel_applicable = ["IB Trader", "Market Maker", "LR Retail Investor"]
                    if bot["Profile"] in tree1_cancel_applicable:
                        bot = cancel_faraway_orders(bot)

                    # Run during the market setup
                    if key_figs.key_figs_test == 'fail' or key_figs.key_figs_test == 'o_semipass':
                        action, state, df_decisions = bl.setup_bot_decision(bot, IB_market_state, df_decisions)
                        if state == 'active' and action != "no_decision":
                            bot = bot_market_interact_initial(action, bot)
                    # Runs during regular market simulation
                    elif key_figs.key_figs_test == 'pass':
                        update_frontend()
                        update_user() 
                        # This checks to see if the bot has no assets or capital available to place an order
                        if bot["Asset"] <= 0 and bot["Wealth"] <= 0:                            # if not assets or capital are available, the bot will sit out, and continue to sitout unless an open order they have is executed or cancelled
                            bot["Delay"] = 100
                            state == 'inactive'
                            action == 'no_decision'
                        elif bot["Asset"] <= 0 and bot["Wealth"] > 0:                           # if no assets are left, then the bot will be forced to buy some assets
                            action = "buy_execute"
                            state = 'active'
                            fp_flag = False
                            liquidity_flag = False
                            emotion_bias = "negative"
                        elif bot["Wealth"] <= 0 and bot["Asset"] > 0:                           # if no capital is left, then the bot will be forced to sell some assets
                            action = "sell_execute"
                            state = 'active'
                            fp_flag = False
                            liquidity_flag = False
                            emotion_bias = "negative"
                        else:                                                                   # regular decision trees run, depeneding on the profile category the bot falls into
                            # Calculates the new risk level based on a variety of conditions
                            bot, risk_df = dr.risk_calculation(bot, buy_orderbook, sell_orderbook, key_figs, transaction_log, risk_df) 
                            # Bot decision trees
                            if bot["Profile"] == "IB Trader":
                                action, state, fp_flag, df_decisions = bl.IB_bot_decision(bot, IB_market_state, key_figs, transaction_log, buy_orderbook, sell_orderbook, df_decisions)
                            elif bot["Profile"] == "WM Trader":
                                action, state, df_decisions = bl.WM_bot_decision(bot, IB_market_state, key_figs, transaction_log, df_decisions)
                            elif bot["Profile"] == "Market Maker":
                                action, state, liquidity_flag, df_decisions = bl.MM_bot_decision(bot, key_figs, buy_orderbook, sell_orderbook, transaction_log, df_decisions)
                            elif bot["Profile"] == "HR Retail Investor" or bot["Profile"] == "LR Retail Investor":
                                action, state, emotion_bias, df_decisions = bl.RI_bot_decision(bot, RI_market_state, key_figs, transaction_log, df_decisions)
                            elif bot["Profile"] == "HR Private Investor" or bot["Profile"] == "LR Private Investor":
                                action, state, df_decisions = bl.PI_bot_decision(bot, PI_market_state, key_figs, df_decisions)
                        
                        # bots that decide to place an order go on to decide the price & qty of the order. The for loop enables multiple orders at once.
                        if state == 'active' and action != "no_decision":
                            timestamp = sim_to_real_time(dt.datetime.now())
                            if bot["Profile"] == "IB Trader":
                                bot, input_orders = bo.IB_order(action, bot, key_figs, fp_flag, timestamp)
                            elif bot["Profile"] == "WM Trader":
                                bot, input_orders = bo.WM_order(action, bot, key_figs, timestamp)
                            elif bot["Profile"] == "Market Maker":
                                bot, input_orders = bo.MM_order(action, bot, key_figs, liquidity_flag, timestamp)
                            elif bot["Profile"] == "HR Retail Investor" or bot["Profile"] == "LR Retail Investor":
                                bot, input_orders = bo.RI_order(action, bot, key_figs, emotion_bias, timestamp)
                            elif bot["Profile"] == "HR Private Investor" or bot["Profile"] == "LR Private Investor":
                                bot, input_orders = bo.PI_order(action, bot, key_figs, timestamp)
                            bot = bot_debiting(bot, input_orders)
                            for index, order in input_orders.iterrows():
                                bot = orderbook_matching(order, bot)
                    bot = timer_reset(bot)
                            
                # merging the updated active bots with the original dataframe
                if len(df_available) > 0:
                    merged_df = df_participants.merge(df_available, on="Trader_ID", how='left', suffixes   =('_old', '_new'))
                    merged_df['Asset'] = merged_df['Asset_new'].fillna(merged_df['Asset_old']).infer_objects(copy=False)
                    merged_df['Wealth'] = merged_df['Wealth_new'].fillna(merged_df['Wealth_old']).infer_objects(copy=False)
                    merged_df['Risk'] = merged_df['Risk_new'].fillna(merged_df['Risk_old']).infer_objects(copy=False)
                    merged_df['Delay'] = merged_df['Delay_new'].fillna(merged_df['Delay_old']).infer_objects(copy=False)
                    merged_df.drop(['Asset_old', 'Wealth_old', 'Risk_old', 'Delay_old', 'Asset_new', 'Wealth_new', 'Risk_new', 'Delay_new'], axis=1, inplace=True)

                    # ensureing compatible dtypes, then updating
                    merged_df['Trader_ID'] = merged_df['Trader_ID'].astype(int)
                    df_participants.update(merged_df)

                # These two for loops look to remove stale orders, unless they are the users orders
                user_profile = df_participants.loc[df_participants['Profile'] == "User"] 
                user_id = user_profile.iloc[0]['Trader_ID']  
                order_timeout = (pause_duration + 45) * time_scalar

                for index, order in buy_orderbook.iterrows():
                    buy_expiry_check = sim_to_real_time(dt.datetime.now()) - order["Timestamp"]
                    if user_id != order["Trader_ID"] and buy_expiry_check.seconds > order_timeout:
                        cancel_time = sim_to_real_time(dt.datetime.now())
                        h_log_update(order, 'order_cancelled_time',cancel_time)
                        buy_orderbook.drop(index=index, inplace=True)
                        df_participants.loc[df_participants["Trader_ID"] == order["Trader_ID"],"Wealth"] += order["Price"] * order["Quantity"]

                for index, order in sell_orderbook.iterrows():
                    sell_expiry_check = sim_to_real_time(dt.datetime.now()) - order["Timestamp"]
                    if user_id != order["Trader_ID"] and sell_expiry_check.seconds > order_timeout:
                        h_log_update(order, 'order_cancelled_time',cancel_time)
                        sell_orderbook.drop(index=index, inplace=True)
                        df_participants.loc[df_participants["Trader_ID"] == order["Trader_ID"], "Asset"] += order["Quantity"]

                # counting down the delay counter
                df_participants["Delay"] = df_participants["Delay"].apply(lambda x: abs(x - 1))
                df_participants["Wealth"] = df_participants["Wealth"].apply(lambda x: round(x, 2))
            inverse_sim_run_flag.set()

def on_sim_button_change(change):  
    global simulation_state
    new_state = change['new']
    if new_state == "Start":
        start_simulation()
    elif new_state == "Pause":
        inverse_sim_run_flag.wait()
        pause_simulation_flag.set()
        pause_simulation()
    elif new_state == "Resume":
        resume_simulation()
    elif new_state == "End":
        simulation_state = "end"
        stop_simulation_flag.set()
        pause_simulation_flag.set()
        stop_simulation()

#front-end display widgets    
simulation_button.observe(on_sim_button_change, names='value')   

display(sim_setup, simulation_button)
AppLayout(header=market_widget, 
          left_sidebar=b_orderbook_widget, 
          center=center_grid, 
          right_sidebar=s_orderbook_widget, 
          footer=user_grid, 
          pane_widths=[1,6,1],
          pane_heights=['220px','600px','300px'],
          grid_gap='10px')

Accordion(children=(VBox(children=(IntText(value=100, description='Number of computer participants:', style=De…

ToggleButtons(options=('Start', 'Pause', 'Resume', 'End'), value=None)

AppLayout(children=(HTML(value='', layout=Layout(grid_area='header')), GridspecLayout(children=(HTML(value='',…

Simulation started.
Simulation stopped.


## Results Analysis 
This can be run directly after the simulator has ended, and will perform various calculations and provide an output with the results from the simulation. 

In [18]:
# Calculates the individual performance of each bot throughout the simulation duration
start_price = 5
key_figs = market_key_figures()
final_price = key_figs.market_price
r_df_participants = df_participants[df_participants.Profile != "User"]

# Credit back the bots with any orders that are still open
for index, buy_order in buy_orderbook.iterrows():
    buy_order_val = buy_order["Price"] * buy_order["Quantity"]
    r_df_participants.loc[r_df_participants['Trader_ID'] == buy_order['Trader_ID'], 'Wealth'] += buy_order_val

for index, sell_order in sell_orderbook.iterrows():
    r_df_participants.loc[r_df_participants['Trader_ID'] == sell_order['Trader_ID'], 'Asset'] += sell_order['Quantity']

# Calculating final positions
r_df_participants["PreValue"] = (r_df_participants["PreAsset"] * start_price) + r_df_participants["PreWealth"]
r_df_participants["Value"] = (r_df_participants["Asset"] * final_price) + r_df_participants["Wealth"]
r_df_participants["Position"] = r_df_participants['Value'] - r_df_participants['PreValue']

def profit_calculator (row):
    if row["Position"] >= 0:
        flag = True
    elif row["Position"] < 0:
        flag = False
    return flag

r_df_participants['Profit'] = r_df_participants.apply(profit_calculator, axis=1)

print(r_df_participants)

     Trader_ID  Asset   Wealth      Risk  Activity  Delay  \
0            0  923.0  6689.65  0.095075  0.935236     32   
1            1  558.0  6855.66  0.739595  0.806192     38   
2            2  891.0  7442.67  0.214327  0.806678      8   
3            3  482.0  8219.15  0.342664  0.853222     56   
4            4  467.0  5436.58  0.458991  0.909454     22   
..         ...    ...      ...       ...       ...    ...   
96          96  138.0  4045.43  0.813037  0.574851     10   
97          97   52.0  4854.00  0.848217  0.563650     27   
98          98  324.0  3900.00  0.902341  0.637961     25   
99          99  222.0  2381.34  0.800114  0.609866      6   
100        100   78.0  2192.00  0.895497  0.551267     12   

                 Profile  PreAsset  PreWealth  PreValue     Value  Position  \
0              IB Trader     762.0     7710.0   11520.0  11323.11   -196.89   
1              IB Trader     564.0     7116.0    9936.0   9656.82   -279.18   
2              IB Trader     9

In [19]:
# Calculates overall how the market has done
pre_value_total = sum(r_df_participants["PreValue"])
value_total = sum(r_df_participants["Value"])
overall_delta = value_total - pre_value_total
per_delta = (overall_delta/pre_value_total) * 100
print(f"Delta:{overall_delta}, which is: {per_delta}%.")

Delta:-2709.20000000007, which is: -0.34821368896750116%.


In [20]:
# Obtains the best and worst performing bots
profit_condition = r_df_participants["Profit"] == True
df_profit = r_df_participants[profit_condition]
df_loss = r_df_participants[~profit_condition]
df_profit.sort_values(by=["Position"], ascending=False, inplace=True)
df_loss.sort_values(by=["Position"], ascending=True, inplace=True)
print(df_profit.head(5))
print(df_loss.head(5))

    Trader_ID  Asset   Wealth      Risk  Activity  Delay    Profile  PreAsset  \
11         11  885.0  6736.46  0.411153  0.921341      4  IB Trader     990.0   
2           2  891.0  7442.67  0.214327  0.806678      8  IB Trader     960.0   
50         50  903.0  9437.96  0.241412  0.790043     17  IB Trader     930.0   
41         41  820.0  9036.60  0.284769  0.805571     34  IB Trader     896.0   
29         29  785.0  8938.12  0.361171  0.910924      5  IB Trader     914.0   

    PreWealth  PreValue     Value  Position  Profit  
11     6208.0   11158.0  11179.16     21.16    True  
2      7096.0   11896.0  11915.49     19.49    True  
50     9302.0   13952.0  13971.02     19.02    True  
41     8654.0   13134.0  13153.00     19.00    True  
29     8290.0   12860.0  12878.82     18.82    True  
    Trader_ID  Asset   Wealth      Risk  Activity  Delay    Profile  PreAsset  \
34         34  494.0  8144.38  0.783929  0.863281     30  IB Trader     788.0   
24         24  600.0  7670.

In [21]:
# Grouping by Profile type, and assessing the aggregate performance of each group
df_category = pd.DataFrame()
bot_profiles = ["IB Trader", "WM Trader", "Market Maker", "HR Retail Investor", "LR Retail Investor", "HR Private Investor", "LR Private Investor"]
for profile in bot_profiles:
    category_condition = r_df_participants["Profile"] == profile
    df_temp = r_df_participants[category_condition]
    pre_val = sum(df_temp["PreValue"])
    val = sum(df_temp["Value"])
    delta = val - pre_val
    per = (delta/pre_val)*100
    profile_performance = pd.Series({
        "Profile": profile,
        "PreValue": pre_val,
        "Value": val, 
        "Overall Delta": delta,
        "Percentage Delta": per
    })
    df_category = pd.concat([df_category, profile_performance.to_frame().T], ignore_index=True)
df_category.sort_values(by=["Percentage Delta"], ascending=False, inplace=True)
df_category

,Profile,PreValue,Value,Overall Delta,Percentage Delta
1,WM Trader,81738.0,81833.87,95.87,0.117289
5,HR Private Investor,23976.0,24001.99,25.99,0.1084
6,LR Private Investor,21442.0,21459.05,17.05,0.079517
2,Market Maker,18550.0,18564.6,14.6,0.078706
4,LR Retail Investor,46486.0,46515.47,29.47,0.063395
3,HR Retail Investor,38762.0,38785.3,23.3,0.06011
0,IB Trader,547074.0,544158.52,-2915.48,-0.532922
